In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Unbiased_news/python_modules')

In [3]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 1.3MB 7.7MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 2.9MB 17.9MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=bc0ebc7b46c1689b788a027d535021fc3e4d8939b8933a040208c1c28083b3e9
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=10504ffab7b60686c5adfd47d909677f565c329d963a60a9c35eb354545340e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict 
from ast import literal_eval
from collections import Counter
import re
import unicodedata
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import copy

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [4]:
# df_sentences = pd.read_csv("/content/drive/MyDrive/Unbiased_news/Data/sent_expanded_ready_for_modeling_v2.csv")

df_sentences = pd.read_csv("Data/sent_expanded_ready_for_modeling_v2.csv")
df_sentences.head()

,article,number,global_bias,title,date,news_title,news_link,bias,news_source,text_ascii,text_sent,text_count
0,0,5,From the Left,Trump Administration Drops Citizenship Questio...,"July 3rd, 2019",Trump Responds After His Administration Drops ...,https://www.huffpost.com/entry/trump-citizensh...,Left,HuffPost,A very sad time for America when the Supreme C...,A very sad time for America when the Supreme C...,0
1,0,5,From the Left,Trump Administration Drops Citizenship Questio...,"July 3rd, 2019",Trump Responds After His Administration Drops ...,https://www.huffpost.com/entry/trump-citizensh...,Left,HuffPost,A very sad time for America when the Supreme C...,to be asked on the #2020 Census!,1
2,0,5,From the Left,Trump Administration Drops Citizenship Questio...,"July 3rd, 2019",Trump Responds After His Administration Drops ...,https://www.huffpost.com/entry/trump-citizensh...,Left,HuffPost,A very sad time for America when the Supreme C...,He added that he had asked his officials to do...,2
3,0,5,From the Left,Trump Administration Drops Citizenship Questio...,"July 3rd, 2019",Trump Responds After His Administration Drops ...,https://www.huffpost.com/entry/trump-citizensh...,Left,HuffPost,A very sad time for America when the Supreme C...,President Donald Trump spoke out Tuesday on hi...,3
4,0,5,From the Left,Trump Administration Drops Citizenship Questio...,"July 3rd, 2019",Trump Responds After His Administration Drops ...,https://www.huffpost.com/entry/trump-citizensh...,Left,HuffPost,A very sad time for America when the Supreme C...,A very sad time for America when the Supreme C...,4


In [ ]:
df_sentences.shape

(42024, 8)

In [ ]:
# df_expanded = pd.read_csv("/content/drive/MyDrive/Unbiased_news/Data/paras_expanded_ready_for_modeling.csv")
# df_expanded.head()

In [5]:
model = SentenceTransformer("E:\\roberta_large_sentence_transformer")

# model = SentenceTransformer("roberta-large-nli-stsb-mean-tokens")

model.max_seq_length = 512

In [13]:
df_small = pd.read_csv("Data/smaller_sent_df.csv")
df_embeds = pd.read_csv("Data/smaller_sent_embeddings.csv", header = None)

In [14]:
df_sentences[df_sentences.number == 86].news_source

697           Washington Post
698           Washington Post
699           Washington Post
700           Washington Post
701           Washington Post
702           Washington Post
703           Washington Post
704           Washington Post
705           Washington Post
706           Washington Post
707           Washington Post
708           Washington Post
709           Washington Post
710           Washington Post
711           Washington Post
712           Washington Post
713           Washington Post
714           Washington Post
715           Washington Post
716           Washington Post
717           Washington Post
718           Washington Post
719           Washington Post
720           Washington Post
721           Washington Post
722           Washington Post
723           Washington Post
724           Washington Post
725           Washington Post
726           Washington Post
727           Washington Post
728           Washington Post
729           Washington Post
730       

In [35]:
df_sentences[df_sentences.news_source == 'HuffPost'].groupby('number')['text_count'].last().describe()

count    122.000000
mean      23.377049
std       21.581032
min        0.000000
25%       11.000000
50%       17.000000
75%       29.000000
max      161.000000
Name: text_count, dtype: float64

In [30]:
print(df_sentences[(df_sentences.number == 3108) & (df_sentences.global_bias == 'From the Left')].iloc[0])

article                                                      930
number                                                      3108
global_bias                                        From the Left
title          Trump Picks State Department Spokeswoman Heath...
date                                          December 7th, 2018
news_title     Trump Appoints State Department’s Heather Naue...
news_link      https://www.huffingtonpost.com/entry/healther-...
bias                                                        Left
news_source                                             HuffPost
text_ascii     State Department spokeswoman Heather Nauert ha...
text_sent      State Department spokeswoman Heather Nauert ha...
text_count                                                     0
Name: 36199, dtype: object


In [28]:
print(df_sentences[(df_sentences.number == 3108) & (df_sentences.global_bias == 'From the Right')].text_ascii.iloc[0])

WASHINGTON  President Donald Trump announced he will nominate State Department spokeswoman Heather Nauert to be the next ambassador to the United Nations.

"I am pleased to announce that Heather Nauert, Spokeswoman for the United States Department of State, will be nominated to serve as United Nations Ambassador. I want to congratulate Heather, and thank Ambassador Nikki Haley for her great service to our Country!" the president tweeted Friday.

The 48-year-old would replace Ambassador Nikki Haley who has been a vocal advocate for US foreign policy  and a strong supporter of Israel at the UN.

It's a tradition Sam Brownback, the US ambassador for International Religious Freedom, is confident Nauert will continue. He points to the time when President Trump moved the US Embassy from Tel Aviv to Jerusalem, noting that Nauert faced the heat for the unpopular decision straight on as State Department spokesman.

"She stood up there and defended the ground because there was a lot of media and

In [39]:
unbias_gen_all(2501)

Smaller Article is 0     Washington Times
1     Washington Times
2     Washington Times
3     Washington Times
4     Washington Times
5     Washington Times
6     Washington Times
7     Washington Times
8     Washington Times
9     Washington Times
10    Washington Times
11    Washington Times
12    Washington Times
13    Washington Times
14    Washington Times
15    Washington Times
16    Washington Times
17    Washington Times
18    Washington Times
19    Washington Times
20    Washington Times
21    Washington Times
Name: news_source, dtype: object
Bigger Article is 0     HuffPost
1     HuffPost
2     HuffPost
3     HuffPost
4     HuffPost
5     HuffPost
6     HuffPost
7     HuffPost
8     HuffPost
9     HuffPost
10    HuffPost
11    HuffPost
12    HuffPost
13    HuffPost
14    HuffPost
15    HuffPost
16    HuffPost
17    HuffPost
18    HuffPost
19    HuffPost
20    HuffPost
21    HuffPost
22    HuffPost
23    HuffPost
24    HuffPost
25    HuffPost
26    HuffPost
27    HuffPost
28  

In [40]:
df_sentences[df_sentences.number == 2501].iloc[0].loc['title']

'Pennsylvania Court Rules Gerrymandered Maps Unconstitutional'

In [22]:
def unbias_gen_all(number):
    
    article_left = df_sentences[(df_sentences.number == number) & (df_sentences.global_bias == 'From the Left')].reset_index(drop = True)
    article_right = df_sentences[(df_sentences.number == number) & (df_sentences.global_bias == 'From the Right')].reset_index(drop = True)

    embeddings_left = model.encode(article_left.text_sent.tolist(), num_workers = 2)
    embeddings_right = model.encode(article_right.text_sent.tolist(), num_workers = 2)

#     article_left = df_sentences[(df_sentences.number == number) & (df_sentences.global_bias == 'From the Left')]
#     article_right = df_sentences[(df_sentences.number == number) & (df_sentences.global_bias == 'From the Right')]
    
#     embeddings_left = df_embeds.loc[article_left.index,:].values
#     embeddings_right = df_embeds.loc[article_right.index,:].values
    
    
    
    left_article_len = len(article_left)
    right_article_len = len(article_right)
    
    if left_article_len >= right_article_len:
        smaller_article = article_right.copy(deep=True)
        smaller_embedding = copy.deepcopy(embeddings_right)
        bigger_article = article_left.copy(deep=True)
        bigger_embedding = copy.deepcopy(embeddings_left)
    else:
        smaller_article = article_left.copy(deep=True)
        smaller_embedding = copy.deepcopy(embeddings_left)
        bigger_article = article_right.copy(deep=True)
        bigger_embedding = copy.deepcopy(embeddings_right)  
    
    smaller_article_html_text = smaller_article.text_ascii.iloc[0]
    bigger_article_html_text = bigger_article.text_ascii.iloc[0]
    
     
    print(f"Smaller Article is {smaller_article.news_source}")
    print(f"Bigger Article is {bigger_article.news_source}")
          
    cosine_scores = np.array(util.pytorch_cos_sim(smaller_embedding, bigger_embedding))

    cos_scores_df = pd.DataFrame(cosine_scores)
    pairs = []

    for row in range(len(smaller_article)):

        i = cos_scores_df.max(axis='columns').idxmax()
        j = cos_scores_df.loc[i].idxmax()

        pairs.append({'index': [i, j], 'score': cos_scores_df.loc[i,j]})

        cos_scores_df.drop(index = i, inplace=True)
        cos_scores_df.drop(columns = j, inplace=True)


    filtered_pairs = []
    counter = 0
    sent_limit = round(len(smaller_article)/2)
    summary_article = []

#     for pair in sorted(pairs, key=lambda x: x['index'][0]):

#         score = pair['score']

#         if score >= 0.55 and counter < round(len(smaller_article)/2):
#             counter += 1
#             filtered_pairs.append(pair)

    
#     filtered_pairs = sorted(pairs, key=lambda x: x['index'][0])
    filtered_pairs = sorted(pairs, key=lambda x: x['score'], reverse = True)
    
    for pair in filtered_pairs:
        print(f"Score: {pair['score']}")
        print(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
        print(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        print('\n')

#         smaller_art_sent_len = len(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
#         bigger_art_sent_len = len(bigger_article.iloc[pair['index'][1]].loc['text_sent'])

#         if smaller_art_sent_len >= bigger_art_sent_len:
#             summary_article.append(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
#         else:
#             summary_article.append(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        
#         start_pos_smaller = smaller_article_html_text.find(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
#         end_pos_smaller = start_pos_smaller + smaller_art_sent_len
        
#         if (start_pos_smaller >= 0) and (end_pos_smaller >= 10):
#             smaller_article_html_text = smaller_article_html_text[:start_pos_smaller] + '<span class="highlight-green">' + \
#                                         smaller_article_html_text[start_pos_smaller:end_pos_smaller] + '</span>' + \
#                                         smaller_article_html_text[end_pos_smaller:]
        
#         start_pos_bigger = bigger_article_html_text.find(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
#         end_pos_bigger = start_pos_bigger + bigger_art_sent_len
        
#         print(start_pos_bigger, end_pos_bigger)
#         if (start_pos_bigger >= 0) and (end_pos_bigger >= 10):
            
#             bigger_article_html_text = bigger_article_html_text[:start_pos_bigger] + '<span class="highlight-green">' + \
#                                     bigger_article_html_text[start_pos_bigger:end_pos_bigger] + '</span>' + \
#                                     bigger_article_html_text[end_pos_bigger:]
        

#     print('\n'.join(summary_article))

# #     st.write()

#     print(f"Article from the left source is published by {article_left.iloc[0].loc['news_source']}")
#     print(f"Titled: {article_left.iloc[0].loc['news_source']}")
    
#     print(smaller_article_html_text)
#     print('\nBigger Article\n')
#     print(bigger_article_html_text)

In [51]:
number_list_size = list(set(df_small.number.tolist()))
# button = st.button("Generate Article Randomly!")

if 1:
    random_number = random.randint(0,len(number_list_size)-1)
    unbias_gen(number_list_size[random_number])

3516 3725
3764 3912
2944 3153
6882 7043
7116 7185
7281 7369
7407 7443
4830 4893
7724 7903
7941 8076
3987 4049
4087 4136
4217 4257
4295 4330
5503 5596
5634 5759
5899 5962
1098 1215
2233 2468
Inside the White House, Woodward portrays an unsteady executive detached from the conventions of governing and prone to snapping at high-ranking staff members, whom he unsettled and belittled on a daily basis.
According to the Post, it portrays a chaotic Trump White House filled with second-guessing by members of Trumps staff, who repeatedly bad-mouth him behind closed doors.
After Trump left the meeting, Woodward recounts, Mattis was particularly exasperated and alarmed, telling close associates that the president acted like  and had the understanding of  a fifth- or sixth-grader.
After Syrian dictator Bashar al-Assad launched a chemical attack on Syrians in April 2017, Trump is said to have demanded Mattis launch an assassination attempt.
Lets kill the fucking lot of them, Trump said, according to

In [21]:
def unbias_gen(number):
    
    # article_left = df_sentences[(df_sentences.number == art_num) & (df_sentences.global_bias == 'From the Left')].reset_index(drop = True)
    # article_right = df_sentences[(df_sentences.number == art_num) & (df_sentences.global_bias == 'From the Right')].reset_index(drop = True)

    # embeddings_left = model.encode(article_left.text_sent.tolist(), num_workers = 2)
    # embeddings_right = model.encode(article_right.text_sent.tolist(), num_workers = 2)

    article_left = df_small[(df_small.number == number) & (df_small.global_bias == 'From the Left')]
    article_right = df_small[(df_small.number == number) & (df_small.global_bias == 'From the Right')]
    
    embeddings_left = df_embeds.loc[article_left.index,:].values
    embeddings_right = df_embeds.loc[article_right.index,:].values
    
    left_article_len = len(article_left)
    right_article_len = len(article_right)
    
    if left_article_len >= right_article_len:
        smaller_article = article_right.copy(deep=True)
        smaller_embedding = copy.deepcopy(embeddings_right)
        bigger_article = article_left.copy(deep=True)
        bigger_embedding = copy.deepcopy(embeddings_left)
    else:
        smaller_article = article_left.copy(deep=True)
        smaller_embedding = copy.deepcopy(embeddings_left)
        bigger_article = article_right.copy(deep=True)
        bigger_embedding = copy.deepcopy(embeddings_right)  
    
    smaller_article_html_text = smaller_article.text_ascii.iloc[0]
    bigger_article_html_text = bigger_article.text_ascii.iloc[0]
   
    cosine_scores = np.array(util.pytorch_cos_sim(smaller_embedding, bigger_embedding))

    cos_scores_df = pd.DataFrame(cosine_scores)
    pairs = []

    for row in range(len(smaller_article)):

        i = cos_scores_df.max(axis='columns').idxmax()
        j = cos_scores_df.loc[i].idxmax()

        pairs.append({'index': [i, j], 'score': cos_scores_df.loc[i,j]})

        cos_scores_df.drop(index = i, inplace=True)
        cos_scores_df.drop(columns = j, inplace=True)


    filtered_pairs = []
    counter = 0
    sent_limit = round(len(smaller_article)/2)
    summary_article = []

    for pair in sorted(pairs, key=lambda x: x['index'][0]):

        score = pair['score']

        if score >= 0.55 and counter < round(len(smaller_article)/2):
            counter += 1
            filtered_pairs.append(pair)


    for pair in filtered_pairs:
        # print(f"Score: {pair['score']}")
        # print(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
        # print(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        # print('\n')

        smaller_art_sent_len = len(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
        bigger_art_sent_len = len(bigger_article.iloc[pair['index'][1]].loc['text_sent'])

        if smaller_art_sent_len >= bigger_art_sent_len:
            summary_article.append(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
        else:
            summary_article.append(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        
        start_pos_smaller = smaller_article_html_text.find(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
        end_pos_smaller = start_pos_smaller + smaller_art_sent_len
        
        if (start_pos_smaller >= 0) and (end_pos_smaller >= 10):
            smaller_article_html_text = smaller_article_html_text[:start_pos_smaller] + '<span class="highlight-green">' + \
                                        smaller_article_html_text[start_pos_smaller:end_pos_smaller] + '</span>' + \
                                        smaller_article_html_text[end_pos_smaller:]
        
        start_pos_bigger = bigger_article_html_text.find(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        end_pos_bigger = start_pos_bigger + bigger_art_sent_len
        
        print(start_pos_bigger, end_pos_bigger)
        if (start_pos_bigger >= 0) and (end_pos_bigger >= 10):
            
            bigger_article_html_text = bigger_article_html_text[:start_pos_bigger] + '<span class="highlight-green">' + \
                                    bigger_article_html_text[start_pos_bigger:end_pos_bigger] + '</span>' + \
                                    bigger_article_html_text[end_pos_bigger:]
        

    print('\n'.join(summary_article))

#     st.write()

    print(f"Article from the left source is published by {article_left.iloc[0].loc['news_source']}")
    print(f"Titled: {article_left.iloc[0].loc['news_source']}")
    
    print(smaller_article_html_text)
    print('\nBigger Article\n')
    print(bigger_article_html_text)

In [8]:
art_num = 695

article_left = df_sentences[(df_sentences.number == art_num) & (df_sentences.global_bias == 'From the Left')].reset_index(drop = True)
article_right = df_sentences[(df_sentences.number == art_num) & (df_sentences.global_bias == 'From the Right')].reset_index(drop = True)

embeddings_left = model.encode(article_left.text_sent.tolist(), num_workers = 2)
embeddings_right = model.encode(article_right.text_sent.tolist(), num_workers = 2)

left_article_len = len(article_left)
right_article_len = len(article_right)

if left_article_len >= right_article_len:
    smaller_article = article_right.copy(deep=True)
    smaller_embedding = copy.deepcopy(embeddings_right)
    bigger_article = article_left.copy(deep=True)
    bigger_embedding = copy.deepcopy(embeddings_left)
else:
    smaller_article = article_left.copy(deep=True)
    smaller_embedding = copy.deepcopy(embeddings_left)
    bigger_article = article_right.copy(deep=True)
    bigger_embedding = copy.deepcopy(embeddings_right)  

cosine_scores = np.array(util.pytorch_cos_sim(smaller_embedding, bigger_embedding))

cos_scores_df = pd.DataFrame(cosine_scores)
pairs = []

for row in range(len(smaller_article)):

    i = cos_scores_df.max(axis='columns').idxmax()
    j = cos_scores_df.loc[i].idxmax()
    cos_scores_df.loc[i,j]

    pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

    cos_scores_df.drop(index = i, inplace=True)
    cos_scores_df.drop(columns = j, inplace=True)


filtered_pairs = []
counter = 0
sent_limit = round(len(smaller_article)/2)
summary_article = []

for pair in sorted(pairs, key=lambda x: x['index'][0]):

  score = pair['score']

    if score >= 0.55 and counter < round(len(smaller_article)/2):
        counter += 1
        filtered_pairs.append(pair)

print(filtered_pairs)
print(len(filtered_pairs))

for pair in filtered_pairs:
#   print(f"Score: {pair['score']}")
#   print(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
#   print(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
#   print('\n')

    smaller_art_sent_len = len(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
    bigger_art_sent_len = len(bigger_article.iloc[pair['index'][1]].loc['text_sent'])

    if smaller_art_sent_len >= bigger_art_sent_len:
        summary_article.append(smaller_article.iloc[pair['index'][0]].loc['text_sent'])
    else:
        summary_article.append(bigger_article.iloc[pair['index'][1]].loc['text_sent'])
        
    

print('\n'.join(summary_article))

[{'index': [0, 6], 'score': 0.69955814}, {'index': [1, 86], 'score': 0.57866657}, {'index': [3, 7], 'score': 0.61656064}, {'index': [4, 113], 'score': 0.85582376}, {'index': [6, 0], 'score': 0.72801536}, {'index': [7, 112], 'score': 0.6822271}, {'index': [8, 71], 'score': 0.591888}, {'index': [9, 70], 'score': 0.728008}, {'index': [11, 36], 'score': 0.8889369}, {'index': [13, 73], 'score': 0.751986}, {'index': [14, 103], 'score': 0.5679965}, {'index': [15, 146], 'score': 0.5668317}, {'index': [18, 88], 'score': 0.6094951}, {'index': [21, 123], 'score': 0.61919296}, {'index': [22, 53], 'score': 0.58941305}, {'index': [26, 15], 'score': 0.64812404}, {'index': [27, 39], 'score': 0.66242236}, {'index': [28, 68], 'score': 0.55178547}, {'index': [30, 107], 'score': 0.6273023}, {'index': [32, 3], 'score': 0.5629289}, {'index': [33, 127], 'score': 0.6480314}, {'index': [41, 142], 'score': 0.69595677}, {'index': [43, 29], 'score': 0.8006475}, {'index': [45, 30], 'score': 0.8670775}, {'index': [